We begin by importing the libs we are going to use:

* The standard [math](https://docs.python.org/3/library/math.html) module provides access to the mathematical functions.
* The [NumPy](https://numpy.org/) lib is fundamental for any kind of scientific computing with Python.
* [pandas](https://pandas.pydata.org/) is a must-have tool for data analysis and manipulation.
* [matplotlib](https://matplotlib.org/) is the most complete package in Python when it comes to data visualizations.
* [seaborn](https://seaborn.pydata.org/) is based on matplotlib as a higher-level set of visualization tools, not as powerful as matplotlib, but much easier to work with and delivers a lot with less work.


In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

Since we have tabular data, we are going to use _pandas_ to load the data and take a first look at it.

To load the data, since the format is CSV (Comma-Separated Values), we use the `read_csv()` function from pandas.

Then we print its shape, which is 1168x81, meaning we have 1168 rows (records) and 81 columns (features).

Actually, we have 1169 rows in the CSV file, but the header that describes the columns doesn't count.

And we actually have 79 features since one of the columns is `SalePrice`, which is the column we will try to predict in a model, and we also will not use the column `Id` and will get rid of it later.

In [ ]:
train = pd.read_csv('raw_data.csv')
train.shape

First, I recommend you to read [this brief description of each column](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

Using the `head()` function from pandas with an argument of 3, we can take a look at the first 3 records.

The `.T` means _Transpose_, this way we visualize rows as columns and vice-versa.

Notice how it doesn't show all of the columns in the middle and only displays `...` because there are too many of them.

In [ ]:
train.head(3).T

The `info()` method from pandas will give you a summary of the data.

Notice how `Alley` has 70 non-null values, meaning it doesn't have a value for most of the 1168 records.

We can also visualize the data types.


In [ ]:
train.info()

The `describe()` method is good to have the first insights of the data.

It automatically gives you descriptive statistics for each feature: number of non-NA/null observations, _mean_, _standard deviation_, the _min_ value, the _quartiles_, and the _max_ value.

Note that the calculations don't take `NaN` values into consideration.

For `LotFrontage`, for instance, it uses only the 964 non-null values, and excludes the other 204 null observations.

## Data Cleaning

In this section, we will perform some Data Cleaning.

### The `id` column

The `id` column is only a dumb identification with no correlation to `SalePrice`.

So let's remove the `id`:


In [ ]:
train.describe().T

In [ ]:
train.drop(columns=['Id'], inplace=True)

### Missing values

When we used `info()` to see the data summary, we could see many columns had a bunch of missing data.

Let's see which columns have missing values and the proportion in each one of them.

`isna()` from pandas will return the missing values for each column, then the `sum()` function will add them up to give you a total.


In [ ]:
columns_with_miss = train.isna().sum()
#filtering only the columns with at least 1 missing value
columns_with_miss = columns_with_miss[columns_with_miss!=0]
#The number of columns with missing values
print('Columns with missing values:', len(columns_with_miss))
#sorting the columns by the number of missing values descending
columns_with_miss.sort_values(ascending=False)

Out of 80 columns, 19 have missing values. 

Missing values per se it not a big problem, but columns with a high number of missing values can cause distortions.

This is the case for:

* PoolQC: Pool quality
* MiscFeature: Miscellaneous feature not covered in other categories
* Alley: Type of alley access to property
* Fence: Fence quality

Let's drop them from the dataset for now.


In [ ]:
# Removing columns
train.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence'], inplace=True)

FireplaceQu has 551 missing values, which is also pretty high.

In this case, the missing values have meaning, which is "NO Fireplace".

Fireplace has the following categories:

* Ex Excellent - Exceptional Masonry Fireplace
* Gd Good - Masonry Fireplace in main level
* TA Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
* Fa Fair - Prefabricated Fireplace in basement
* Po Poor - Ben Franklin Stove
* NA No Fireplace

Let's check the correlation between FireplaceQu and SalePrice, to see how important this feature is in order to determine the price.

First, we will replace the missing values for 0.

Then, we encode the categories into numbers from 1 to 5.


In [ ]:
train['FireplaceQu'].fillna(0, inplace=True)
train['FireplaceQu'].replace({'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}, inplace=True)

Using a barplot, we can see how the category of the FirePlace increases the value of SalePrice.

It is also worth noting how much higher the value is when the house has an Excellent fireplace.

This means we should keep FireplaceQu as feature.



In [ ]:
sns.set(style="whitegrid")
sns.barplot(x='FireplaceQu', y="SalePrice", data=train)

### Missing values in numeric columns

Another feature with a high number of missing values is LotFrontage with a count 204.

Let’s see the correlation between the remaining features with missing values and the SalePrice.


In [ ]:
columns_with_miss = train.isna().sum()
columns_with_miss = columns_with_miss[columns_with_miss!=0]
c = list(columns_with_miss.index)
c.append('SalePrice')
train[c].corr()

Note that LotFrontage, MasVnrArea, and GarageYrBlt have a positive correlation with SalePrice, but this correlation isn't very strong.

To simplify this analisys, we will remove theses columns for now:

In [ ]:
cols_to_be_removed = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea']
train.drop(columns=cols_to_be_removed, inplace=True)

Finally, these are the remaining columns with missing values:


In [ ]:
columns_with_miss = train.isna().sum()
columns_with_miss = columns_with_miss[columns_with_miss!=0]
print(f'Columns with missing values: {len(columns_with_miss)}')
columns_with_miss.sort_values(ascending=False)

## Categorical variables

Let's work on the categorical variables of our dataset.

### Dealing with missing values

Filling Categorical NaN that we know how to fill due to the [description file](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=data_description.txt).

In [ ]:
# Fills NA in place of NaN
for c in ['GarageType', 'GarageFinish', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1']:
    train[c].fillna('NA', inplace=True)
    
# Fills None in place of NaN
train['MasVnrType'].fillna('None', inplace=True)

With this have only 5 columns with missing values left in our dataset.


In [ ]:
columns_with_miss = train.isna().sum()
columns_with_miss = columns_with_miss[columns_with_miss!=0]
print(f'Columns with missing values: {len(columns_with_miss)}')
columns_with_miss.sort_values(ascending=False)

### Ordinal

Also by reading the [description file](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=data_description.txt), we can identify other variables that have a similar system to FireplaceQu to categorize the quality: Poor, Good, Excellent, etc.

We are going to replicate the treatment we gave to FireplaceQu to these variables according to the following descriptions:

ExterQual: Evaluates the quality of the material on the exterior

* Ex Excellent
* Gd Good
* TA Average/Typical
* Fa Fair
* Po Poor

ExterCond: Evaluates the present condition of the material on the exterior

* Ex Excellent
* Gd Good
* TA Average/Typical
* Fa Fair
* Po Poor

BsmtQual: Evaluates the height of the basement

* Ex Excellent (100+ inches)
* Gd Good (90-99 inches)
* TA Typical (80-89 inches)
* Fa Fair (70-79 inches)
* Po Poor ( < 70 inches)
* NA No Basement

BsmtCond: Evaluates the general condition of the basement

* Ex Excellent
* Gd Good
* TA Typical - slight dampness allowed
* Fa Fair - dampness or some cracking or settling
* Po Poor - Severe cracking, settling, or wetness
* NA No Basement

HeatingQC: Heating quality and condition

* Ex Excellent
* Gd Good
* TA Average/Typical
* Fa Fair
* Po Poor

KitchenQual: Kitchen quality

* Ex Excellent
* Gd Good
* TA Average/Typical
* Fa Fair
* Po Poor

GarageQual: Garage quality

* Ex Excellent
* Gd Good
* TA Average/Typical
* Fa Fair
* Po Poor
* NA No Garage

GarageCond: Garage condition

* Ex Excellent
* Gd Good
* TA Average/Typical
* Fa Fair
* Po Poor
* NA No Garage



In [ ]:
ord_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond']
for col in ord_cols:
    train[col].fillna(0, inplace=True)
    train[col].replace({'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}, inplace=True)

Let's now plot the correlation of these variables with SalePrice.


In [ ]:
ord_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond']
f, axes = plt.subplots(2, 4, figsize=(15, 10), sharey=True)

for r in range(0, 2):
    for c in range(0, 4):
        sns.barplot(x=ord_cols.pop(), y="SalePrice", data=train, ax=axes[r][c])

plt.tight_layout()
plt.show()

As you can see, the better the category of a variable, the higher the price, which means these variables will be important for a prediction model.


### Nominal

Other categorical variables don't seem to follow any clear ordering.

Let's see how many values these columns can assume:


In [ ]:
cols = train.columns
num_cols = train._get_numeric_data().columns
nom_cols = list(set(cols) - set(num_cols))
print(f'Nominal columns: {len(nom_cols)}')

value_counts = {}
for c in nom_cols:
    value_counts[c] = len(train[c].value_counts())

sorted_value_counts = {k: v for k, v in sorted(value_counts.items(), key=lambda item: item[1])}
sorted_value_counts

Some categorical variables can assume several different values like Neighborhood. 

To simplify, let's analyze only variables with 6 different values or less.


In [ ]:
nom_cols_less_than_6 = []
for c in nom_cols:
    n_values = len(train[c].value_counts())
    if n_values < 7:
        nom_cols_less_than_6.append(c)

print(f'Nominal columns with less than 6 values: {len(nom_cols_less_than_6)}')


Plotting against SalePrice to have a better idea of how they affect it:


In [ ]:
ncols = 3
nrows = math.ceil(len(nom_cols_less_than_6) / ncols)
f, axes = plt.subplots(nrows, ncols, figsize=(15, 30))

for r in range(0, nrows):
    for c in range(0, ncols):
        if not nom_cols_less_than_6:
            continue
        sns.barplot(x=nom_cols_less_than_6.pop(), y="SalePrice", data=train, ax=axes[r][c])

plt.tight_layout()
plt.show()

We can see a good correlation of many of these columns with the target variable.

For now, let's keep them.

We still have NaN in 'Electrical'.

As we could see in the plot above, 'SBrkr' is the most frequent value in 'Electrical'.

Let's use this value to replace NaN in Electrical.


In [ ]:
# Inputs more frequent value in place of NaN

train['Electrical'].fillna('SBrkr', inplace=True)


### Zero values

Another quick check is to see how many columns have lots of data equals to 0.


In [ ]:
train.isin([0]).sum().sort_values(ascending=False).head(25)

In this case, even though there are many 0's, they have meaning.

For instance, PoolArea (Pool area in square feet) equals 0 means that the house doesn't have any pool area.

This is important information correlated to the house and thus, we are going to keep them.

## Saving cleaned data

Let's see how the cleaned data looks like and how many columns we have left.

We have no more missing values:

In [ ]:
columns_with_miss = train.isna().sum()
columns_with_miss = columns_with_miss[columns_with_miss!=0]
print(f'Columns with missing values: {len(columns_with_miss)}')
columns_with_miss.sort_values(ascending=False)

After cleaning the data, we are left with 73 columns out of the initial 81.


In [ ]:
train.shape

Let's take a look at the first 3 records of the cleaned data.


In [ ]:
train.head(3).T

We can see a summary of the data showing that, for all the 1168 records, there isn't a single missing (null) value.

In [ ]:
train.info()

Finally, let's save the cleaned data in a separate file.


In [ ]:
train.to_csv('train-cleaned.csv')

## Conclusions

We dealt with missing values and removed the following columns: 'Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'LotFrontage', 'GarageYrBlt', 'MasVnrArea'.

We also:

* Replaced the NaN with NA in the following columns: 'GarageType', 'GarageFinish', 'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1'.
* Replaced the NaN with None in 'MasVnrType'.
* Imputed the most frequent value in place of NaN in 'Electrical'.

Please note that the removed columns are not useless and may contribute to the final model.

After the first round of analysis and testing of the hypothesis, if you ever need to improve your future model further, you can consider reevaluating these columns and understand them better to see how they fit into the problem.

Data Analysis and Machine Learning is NOT a straight path.

It is a process where you iterate and keep testing ideas until you have the result you want, or until find out the result you need is not possible.

We are going to use this data to create our Machine Learning model and predict the house prices.
